In [2]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
import json
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

g:\yt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give Short, courteous answers. no more than 1 sentence."
system_message += "Always be accurate. If you don't know the answer, say so."

In [42]:
def chat(message,history):
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [43]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI.Give Short, courteous answers. no more than 1 sentence.Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'content': 'hi'}]


In [55]:
MODEL_OPTIONS = {
    "OpenAI GPT-4o-mini": "openai:gpt-4o-mini",
    "LLAMA3.2 (Ollama)": "ollama:llama3.2:1b",
    "DeepSeek-R1 1.5B (Ollama)": "ollama:deepseek-r1:1.5b"
}

In [59]:
def get_model_response(messages, model_identifier):
    """
    Gets a streaming response from the selected model.
    This function now yields individual tokens for a smooth UI update.
    """
    provider, model_name = model_identifier.split(":", 1)

    if provider == "openai":
        client = OpenAI() # Corrected from OpenAI()
        stream = client.chat.completions.create(
            model=model_name,
            messages=messages,
            stream=True
        )
        for chunk in stream:
            token = chunk.choices[0].delta.content or ""
            yield token

    elif provider == "ollama":
        url = "http://localhost:11434/api/chat"
        payload = {
            "model": model_name,
            "messages": messages,
            "stream": True
        }
        try:
            with requests.post(url, json=payload, stream=True) as r:
                r.raise_for_status()
                for line in r.iter_lines():
                    if line:
                        chunk = json.loads(line.decode("utf-8"))
                        token = chunk.get("message", {}).get("content", "")
                        yield token
        except requests.exceptions.RequestException as e:
            yield f"Error: Could not connect to Ollama. {e}"

In [60]:
def chat_wrapper(message,history,model_choice):
    app_history = [{'role':'system','content':system_message}]
    for user_msg,ai_msg in history:
        app_history.append({'role':'user','content':user_msg})
        if ai_msg:  
            app_history.append({'role':'assistant','content':ai_msg})

    app_history.append({'role':'user','content':message})

    api_model_identifier = MODEL_OPTIONS[model_choice]

    history.append((message,""))

    responss_stream = get_model_response(app_history,api_model_identifier)

    full_response = ""
    for token in responss_stream:
        full_response += token
        history[-1] = (message,full_response)
        yield history

    


In [61]:
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("FlightAI Chatbot")

    model_dropdown = gr.Dropdown(
        choices=list(MODEL_OPTIONS.keys()),
        value="OpenAI GPT-4o-mini",
        label="Select Model"
    )

    chatbot = gr.Chatbot(
        [],
        elem_id='chatbot',
        label='chat',
        bubble_full_width=False,
        height=500
    )

    textbox = gr.Textbox(
        placeholder="Ask something about FlightAI",
        label="Enter your message",
        scale=7
    )

    textbox.submit(
        fn=chat_wrapper,
        inputs = [textbox,chatbot,model_dropdown],
        outputs=[chatbot]
    )

    textbox.submit(lambda:"",inputs=[],outputs=[textbox])

if __name__=="__main__":
    app.launch()


C:\Users\ankit\AppData\Local\Temp\ipykernel_2288\839480612.py:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\ankit\AppData\Local\Temp\ipykernel_2288\839480612.py:10: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.
